# Hyperparameter Tuning using Keras

In this notebook, we learn how to tune a set of hyperparameters for an Artificial Neural Network (ANN) using Keras.

ANNs are highly flexible. This comes with a cost. There could be many hyperparameters to tune. Eevn in simple Multi-Layer Perceptron (MLP) we can change the number of hidden layers, neurons, type of activation functions, weight initializer, regularizers, etc.

We can perform K-fold cross-validation by using Scikit-Learn's GridSearchCV or RandomizedSearchCV. To do this, we need to wrap our Keras models in objects that mimic regular Scikit-Learn classifiers or regressors. The first step is to create a function that will build and compile a Keras model, given a set of hyperparameters (that we wish to tune)

We conduct a simple illustrative experiment to tune two hyperparameters for a fixed architecture MLP on the MNIST handwritten digits dataset: learning rate and momentum for the SGD (Stochastic Gradient Descent) optimizer

In [1]:
import warnings
import os
import time
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_val_predict


import tensorflow as tf
from tensorflow import keras

## Load the Dataset

We load the dataset using Keras. Then, create train, test and validation subset. Finally, scale the data.

In [2]:
mnist = keras.datasets.mnist

(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()


X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.0

print("No. of Training Samples: ", X_train.shape)
print("No. of Training Labels: ", y_train.shape)

print("\nNo. of Validation Samples: ", X_valid.shape)
print("No. of Validation Labels: ", y_valid.shape)

print("\nNo. of Testing Samples: ", X_test.shape)
print("No. of Testing Labels: ", y_test.shape)

print("\nX type: ", X_train.dtype)
print("y type: ", y_train.dtype)

No. of Training Samples:  (55000, 28, 28)
No. of Training Labels:  (55000,)

No. of Validation Samples:  (5000, 28, 28)
No. of Validation Labels:  (5000,)

No. of Testing Samples:  (10000, 28, 28)
No. of Testing Labels:  (10000,)

X type:  float64
y type:  uint8


## A function to Build and Compile a Keras Model

Note that we use only two hyperparameters as arguments of the function that we want to tune.

It is good practice to provide reasonable defaults to the hyperparameters, as Scikit-Learn does.

In [3]:
def build_model(sgd_lrate=3e-3, sgd_momentum=0.1):
    
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=[28, 28]))
    model.add(keras.layers.Dense(300, activation="relu"))
    model.add(keras.layers.Dense(100, activation="relu"))
    model.add(keras.layers.Dense(10, activation="softmax"))
    
    optimizer = keras.optimizers.SGD(learning_rate=sgd_lrate, momentum=sgd_momentum)
    
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=optimizer,
                  metrics=["accuracy"])
    
    return model

## Implementation of the Scikit-Learn Classifier API for Keras

We create a KerasClassifier object. It is a thin wrapper around the Keras model built using build_model(). 

It will use the default hyperparameters we defined in build_model(). 

We can use this object like a regular Scikit-Learn classifier.

In [4]:
keras_classifier = tf.keras.wrappers.scikit_learn.KerasClassifier(build_model)

## Hyperparameter Tuning

For hyperparameter tuning we create a GridSearchCV object using the KerasClassifier and a set of hyperparameters with their list of values.

In the fit() method of the GridSearchCV object we pass some parameters for the underlying Keras model. 

Since the GridSearchCV uses K-fold cross-validation, it does not use X_valid and y_valid in the argument of the fit() method. Those are only used for early stopping.

In [5]:
%%time

warnings.filterwarnings('ignore')

# The param_grid tells Scikit-Learn to evaluate all combinations of the hyperparameter values
param_grid = {'sgd_lrate': [1e-2, 1e-1, 2e-1], 
              'sgd_momentum': [0.1, 0.3, 0.5]}


early_stopping_cb = keras.callbacks.EarlyStopping(monitor='val_loss',
                                  min_delta=0, 
                                  patience=10, 
                                  verbose=0, 
                                  mode='auto',
                                  restore_best_weights=True)

mlp_cv = GridSearchCV(keras_classifier, param_grid, scoring='accuracy', cv=5, verbose=1)
mlp_cv.fit(X_train, y_train, 
           epochs=100,
           validation_data=(X_valid, y_valid),
           callbacks=[early_stopping_cb])


params_optimal_mlp = mlp_cv.best_params_

print("Best Score: %f" % mlp_cv.best_score_)
print("Optimal Hyperparameter Values: ", params_optimal_mlp)
print("\n")

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Train on 44000 samples, validate on 5000 samples
Epoch 1/100
44000/44000 [==============================] - 3s 77us/sample - loss: 0.6326 - accuracy: 0.8376 - val_loss: 0.3262 - val_accuracy: 0.9126
Epoch 2/100
44000/44000 [==============================] - 3s 66us/sample - loss: 0.2993 - accuracy: 0.9147 - val_loss: 0.2533 - val_accuracy: 0.9314
Epoch 3/100
44000/44000 [==============================] - 3s 65us/sample - loss: 0.2450 - accuracy: 0.9311 - val_loss: 0.2165 - val_accuracy: 0.9398
Epoch 4/100
44000/44000 [==============================] - 3s 67us/sample - loss: 0.2116 - accuracy: 0.9407 - val_loss: 0.1961 - val_accuracy: 0.9452
Epoch 5/100
44000/44000 [==============================] - 3s 67us/sample - loss: 0.1861 - accuracy: 0.9475 - val_loss: 0.1744 - val_accuracy: 0.9514
Epoch 6/100
44000/44000 [==============================] - 3s 65us/sample - loss: 0.1665 - accuracy: 0.9532 - val_loss: 0.1589 - val_accuracy: 0.9556
Epoch 7/100
44000/44000 [==========================

Epoch 55/100
44000/44000 [==============================] - 3s 65us/sample - loss: 0.0086 - accuracy: 0.9993 - val_loss: 0.0747 - val_accuracy: 0.9790
Epoch 56/100
44000/44000 [==============================] - 3s 65us/sample - loss: 0.0082 - accuracy: 0.9993 - val_loss: 0.0753 - val_accuracy: 0.9786
Epoch 57/100
44000/44000 [==============================] - 3s 65us/sample - loss: 0.0078 - accuracy: 0.9995 - val_loss: 0.0733 - val_accuracy: 0.9794
Train on 44000 samples, validate on 5000 samples
Epoch 1/100
44000/44000 [==============================] - 3s 75us/sample - loss: 0.6382 - accuracy: 0.8320 - val_loss: 0.3170 - val_accuracy: 0.9166
Epoch 2/100
44000/44000 [==============================] - 3s 66us/sample - loss: 0.3008 - accuracy: 0.9140 - val_loss: 0.2532 - val_accuracy: 0.9302
Epoch 3/100
44000/44000 [==============================] - 3s 66us/sample - loss: 0.2459 - accuracy: 0.9286 - val_loss: 0.2114 - val_accuracy: 0.9438
Epoch 4/100
44000/44000 [=======================

Epoch 3/100
44000/44000 [==============================] - 3s 65us/sample - loss: 0.2467 - accuracy: 0.9295 - val_loss: 0.2219 - val_accuracy: 0.9394
Epoch 4/100
44000/44000 [==============================] - 3s 65us/sample - loss: 0.2113 - accuracy: 0.9405 - val_loss: 0.1907 - val_accuracy: 0.9474
Epoch 5/100
44000/44000 [==============================] - 3s 66us/sample - loss: 0.1860 - accuracy: 0.9467 - val_loss: 0.1723 - val_accuracy: 0.9518
Epoch 6/100
44000/44000 [==============================] - 3s 65us/sample - loss: 0.1650 - accuracy: 0.9522 - val_loss: 0.1620 - val_accuracy: 0.9558
Epoch 7/100
44000/44000 [==============================] - 3s 65us/sample - loss: 0.1486 - accuracy: 0.9579 - val_loss: 0.1452 - val_accuracy: 0.9608
Epoch 8/100
44000/44000 [==============================] - 3s 65us/sample - loss: 0.1346 - accuracy: 0.9622 - val_loss: 0.1360 - val_accuracy: 0.9644
Epoch 9/100
44000/44000 [==============================] - 3s 65us/sample - loss: 0.1229 - accuracy:

...

Epoch 5/100
44000/44000 [==============================] - 3s 67us/sample - loss: 0.0354 - accuracy: 0.9884 - val_loss: 0.0795 - val_accuracy: 0.9776
Epoch 6/100
44000/44000 [==============================] - 3s 67us/sample - loss: 0.0262 - accuracy: 0.9916 - val_loss: 0.0765 - val_accuracy: 0.9794
Epoch 7/100
44000/44000 [==============================] - 3s 64us/sample - loss: 0.0193 - accuracy: 0.9939 - val_loss: 0.0773 - val_accuracy: 0.9792
Epoch 8/100
44000/44000 [==============================] - 3s 67us/sample - loss: 0.0156 - accuracy: 0.9948 - val_loss: 0.0787 - val_accuracy: 0.9802
Epoch 9/100
44000/44000 [==============================] - 3s 64us/sample - loss: 0.0098 - accuracy: 0.9971 - val_loss: 0.0959 - val_accuracy: 0.9770
Epoch 10/100
44000/44000 [==============================] - 3s 64us/sample - loss: 0.0073 - accuracy: 0.9980 - val_loss: 0.0791 - val_accuracy: 0.9828
Epoch 11/100
44000/44000 [==============================] - 3s 64us/sample - loss: 0.0039 - accurac

Epoch 3/100
44000/44000 [==============================] - 3s 64us/sample - loss: 0.0663 - accuracy: 0.9793 - val_loss: 0.0870 - val_accuracy: 0.9746
Epoch 4/100
44000/44000 [==============================] - 3s 64us/sample - loss: 0.0476 - accuracy: 0.9847 - val_loss: 0.0762 - val_accuracy: 0.9800
Epoch 5/100
44000/44000 [==============================] - 3s 64us/sample - loss: 0.0321 - accuracy: 0.9893 - val_loss: 0.0844 - val_accuracy: 0.9794
Epoch 6/100
44000/44000 [==============================] - 3s 64us/sample - loss: 0.0270 - accuracy: 0.9910 - val_loss: 0.0744 - val_accuracy: 0.9798
Epoch 7/100
44000/44000 [==============================] - 3s 65us/sample - loss: 0.0202 - accuracy: 0.9936 - val_loss: 0.0762 - val_accuracy: 0.9802
Epoch 8/100
44000/44000 [==============================] - 3s 64us/sample - loss: 0.0107 - accuracy: 0.9966 - val_loss: 0.0737 - val_accuracy: 0.9820
Epoch 9/100
44000/44000 [==============================] - 3s 63us/sample - loss: 0.0104 - accuracy:

44000/44000 [==============================] - 3s 64us/sample - loss: 0.0016 - accuracy: 0.9995 - val_loss: 0.0969 - val_accuracy: 0.9842
Epoch 14/100
44000/44000 [==============================] - 3s 64us/sample - loss: 0.0014 - accuracy: 0.9997 - val_loss: 0.0929 - val_accuracy: 0.9844
Epoch 15/100
44000/44000 [==============================] - 3s 65us/sample - loss: 2.3315e-04 - accuracy: 1.0000 - val_loss: 0.0933 - val_accuracy: 0.9844
Epoch 16/100
44000/44000 [==============================] - 3s 65us/sample - loss: 1.3771e-04 - accuracy: 1.0000 - val_loss: 0.0946 - val_accuracy: 0.9844
Epoch 17/100
44000/44000 [==============================] - 3s 64us/sample - loss: 1.1179e-04 - accuracy: 1.0000 - val_loss: 0.0958 - val_accuracy: 0.9844
Epoch 18/100
44000/44000 [==============================] - 3s 64us/sample - loss: 9.5805e-05 - accuracy: 1.0000 - val_loss: 0.0971 - val_accuracy: 0.9844
Epoch 19/100
44000/44000 [==============================] - 3s 66us/sample - loss: 8.5413e-

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed: 55.3min finished


Train on 55000 samples, validate on 5000 samples
Epoch 1/100
55000/55000 [==============================] - 4s 79us/sample - loss: 0.2409 - accuracy: 0.9278 - val_loss: 0.1112 - val_accuracy: 0.9664
Epoch 2/100
55000/55000 [==============================] - 4s 66us/sample - loss: 0.0984 - accuracy: 0.9698 - val_loss: 0.0861 - val_accuracy: 0.9746
Epoch 3/100
55000/55000 [==============================] - 4s 67us/sample - loss: 0.0666 - accuracy: 0.9797 - val_loss: 0.0731 - val_accuracy: 0.9774
Epoch 4/100
55000/55000 [==============================] - 4s 65us/sample - loss: 0.0480 - accuracy: 0.9848 - val_loss: 0.0746 - val_accuracy: 0.9790
Epoch 5/100
55000/55000 [==============================] - 4s 65us/sample - loss: 0.0333 - accuracy: 0.9898 - val_loss: 0.0741 - val_accuracy: 0.9762
Epoch 6/100
55000/55000 [==============================] - 4s 65us/sample - loss: 0.0264 - accuracy: 0.9916 - val_loss: 0.0777 - val_accuracy: 0.9776
Epoch 7/100
55000/55000 [==========================

## Select The Best Model

Using the optimal hyperparameter values, create the best model.

In [6]:
model = mlp_cv.best_estimator_.model

## Evaluate the Optimal Model

In [7]:
train_loss = model.evaluate(X_train, y_train, verbose=0)[0]
train_accuracy = model.evaluate(X_train, y_train, verbose=0)[1]

test_loss = model.evaluate(X_test, y_test, verbose=0)[0]
test_accuracy = model.evaluate(X_test, y_test, verbose=0)[1]

print("Constant Learning Rate - Train Accuracy: ", train_accuracy)
print("Constant Learning Rate - Test Accuracy: ", test_accuracy)

print("\nConstant Learning Rate - Train Loss: ", train_loss)
print("Constant Learning Rate - Test Loss: ", test_loss)

# model.predict(X_test) method return 10 probabilities per class for each instance (Dimension Nx10)
y_test_predicted = model.predict(X_test)
y_test_predicted = np.argmax(y_test_predicted, axis=1) # get the label/index of the highest probability class

# model.predict_classes(X_test) method returns the index (class label) with largest probability (1D array)
#y_test_predicted= model.predict_classes(X_test)


y_train_predicted = model.predict(X_train)
y_train_predicted = np.argmax(y_train_predicted, axis=1) # get the label/index of the highest probability class


print("\nTest Confusion Matrix:")
print(confusion_matrix(y_test, y_test_predicted))

print("\nClassification Report:")
print(classification_report(y_test, y_test_predicted))

Constant Learning Rate - Train Accuracy:  0.99716365
Constant Learning Rate - Test Accuracy:  0.9806

Constant Learning Rate - Train Loss:  0.011166327119051394
Constant Learning Rate - Test Loss:  0.06529028431936458

Test Confusion Matrix:
[[ 973    1    1    0    1    0    2    1    1    0]
 [   0 1125    3    1    0    0    2    1    3    0]
 [   5    1 1010    5    2    0    2    3    3    1]
 [   0    0    5  995    0    2    0    2    3    3]
 [   1    0    3    1  961    0    1    1    0   14]
 [   2    0    0   14    1  870    2    0    1    2]
 [   4    4    1    1    3    2  941    1    1    0]
 [   1    6    6    2    1    0    1 1003    2    6]
 [   4    1    4    9    2    1    1    3  945    4]
 [   2    2    0    5   11    2    0    3    1  983]]

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       980
           1       0.99      0.99      0.99      1135
           2       0.98      0.98      0